<a href="https://colab.research.google.com/github/anas-awadalla/Incepto/blob/master/interpretability_experiments/MultiFeature_(color%2C_texture%2C_edges%2C_location)_Clustering_HAM10000_Interpretability_Attribution_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --quiet torchray

In [5]:
from torchray.attribution.extremal_perturbation import extremal_perturbation, contrastive_reward
from torchray.benchmark import get_example_data, plot_example
from torchray.utils import get_device
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch

In [6]:
model = torch.load("/content/drive/My Drive/mHealth Privacy/Evaluating Models/Saved Models/kaggle90-best.pth", map_location=torch.device('cuda:0')) 
model.eval()
print(model)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torchvision.models.densenet.DenseNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or 

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [7]:
trans = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor()])

trans_color = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor()])

## Generate Masks

Image Segmentation

In [8]:
import cv2

def remove_background():
    #== Parameters =======================================================================
    BLUR = 5
    CANNY_THRESH_1 = 10
    CANNY_THRESH_2 = 50
    MASK_DILATE_ITER = 20
    MASK_ERODE_ITER = 20
    MASK_COLOR = (0.0,0.0,0.0) # In BGR format

    #== Processing =======================================================================
    img = cv2.imread("img1.png")
    img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)

    #-- Read image -----------------------------------------------------------------------
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    #-- Edge detection -------------------------------------------------------------------
    edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)

    #-- Find contours in edges, sort by area ---------------------------------------------
    contour_info = []
    contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    for c in contours:
        contour_info.append((
            c,
            cv2.isContourConvex(c),
            cv2.contourArea(c),
        ))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)


    #-- Create empty mask, draw filled polygon on it corresponding to largest contour ----
    # Mask is black, polygon is white
    mask = np.zeros(edges.shape)
    for c in contour_info:
        cv2.fillConvexPoly(mask, c[0], (255))
    # cv2.fillConvexPoly(mask, max_contour[0], (255))

    #-- Smooth mask, then blur it --------------------------------------------------------
    mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
    mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
    mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
    mask_stack = np.dstack([mask]*3)    # Create 3-channel alpha mask

    #-- Blend masked img into MASK_COLOR background --------------------------------------
    mask_stack  = mask_stack.astype('float32') / 255.0          # Use float matrices, 
    img         = img.astype('float32') / 255.0                 #  for easy blending

    masked = (mask_stack * img) + ((1 - mask_stack) * MASK_COLOR) # Blend
    masked = (masked * 255).astype('uint8')                     # Convert back to 8-bit 
    cv2.imwrite('img.png', masked)
    return masked

## HAM10000 Dataset

In [9]:
import pandas as pd
import glob
import os

#make a dataset
data = pd.read_csv("/content/drive/My Drive/mHealth Privacy/Evaluating Models/Data/skin-cancer-mnist-ham10000/HAM10000_metadata.csv")

print(data)
base_skin_dir = os.path.join('/content/drive/My Drive/mHealth Privacy/Evaluating Models/Data/skin-cancer-mnist-ham10000/HAM10000_images_total')


lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'dermatofibroma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

data['cell_type'] = data['dx'].map(lesion_type_dict.get) 
data['cell_type_idx'] = pd.Categorical(data['cell_type']).codes

data[['cell_type_idx', 'cell_type']].sort_values('cell_type_idx').drop_duplicates()

data['cell_type'].value_counts()


         lesion_id      image_id     dx dx_type   age     sex localization
0      HAM_0000118  ISIC_0027419    bkl   histo  80.0    male        scalp
1      HAM_0000118  ISIC_0025030    bkl   histo  80.0    male        scalp
2      HAM_0002730  ISIC_0026769    bkl   histo  80.0    male        scalp
3      HAM_0002730  ISIC_0025661    bkl   histo  80.0    male        scalp
4      HAM_0001466  ISIC_0031633    bkl   histo  75.0    male          ear
...            ...           ...    ...     ...   ...     ...          ...
10010  HAM_0002867  ISIC_0033084  akiec   histo  40.0    male      abdomen
10011  HAM_0002867  ISIC_0033550  akiec   histo  40.0    male      abdomen
10012  HAM_0002867  ISIC_0033536  akiec   histo  40.0    male      abdomen
10013  HAM_0000239  ISIC_0032854  akiec   histo  80.0    male         face
10014  HAM_0003521  ISIC_0032258    mel   histo  70.0  female         back

[10015 rows x 7 columns]


Melanocytic nevi                  6705
dermatofibroma                    1113
Benign keratosis-like lesions     1099
Basal cell carcinoma               514
Actinic keratoses                  327
Vascular lesions                   142
Dermatofibroma                     115
Name: cell_type, dtype: int64

In [10]:
from PIL import Image
class dataset(Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, df, transform=None):
        'Initialization'
        self.df = df
        self.transform = transform

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.df)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Load data and get label
        X = Image.open("/content/drive/My Drive/mHealth Privacy/Evaluating Models/Data/skin-cancer-mnist-ham10000/HAM10000_images_total/"+self.df['image_id'][index]+".jpg")
        y = torch.tensor(int(self.df['cell_type_idx'][index]))

        if self.transform:
            X = self.transform(X)

        return X, y

In [11]:
transform=transforms.Compose([
            transforms.Resize((224 ,224)),
            transforms.ToTensor()])

ham_data = dataset(data, transform=transform)
dataloader = DataLoader(ham_data)

## ISIC Dataset

In [98]:
transform=transforms.Compose([
            transforms.Resize((224 ,224)),
            transforms.ToTensor()])
isic_data = torchvision.datasets.ImageFolder("/content/drive/My Drive/mHealth Privacy/Evaluating Models/Data/skin-cancer-malignant-vs-benign/test", transform=transform)

## Find Masks

Get Texture feature

In [12]:
from skimage.feature import local_binary_pattern
from scipy.stats import itemfreq
from sklearn.preprocessing import normalize
import cv2

def compare_texture(src,img):

    radius = 3
    no_points = 8 * radius
    
    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(src_gray, no_points, radius, method='uniform')
    x = np.unique(lbp.ravel(), return_counts=True)
    hist_src = x[:, 1]/sum(x[:, 1])


    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(img_gray, no_points, radius, method='uniform')
    x = np.unique(lbp.ravel(), return_counts=True)
    hist_img = x[:, 1]/sum(x[:, 1])

    return cv2.compareHist(np.array(hist_src, dtype=np.float32), np.array(hist_img, dtype=np.float32), 2)

Get Color feature

In [13]:
import cv2
import numpy as np
from skimage import io

def get_colors(img):
  average = img.mean(axis=0).mean(axis=0)

  pixels = np.float32(img.reshape(-1, 3))

  n_colors = 5
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
  flags = cv2.KMEANS_RANDOM_CENTERS

  _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
  _, counts = np.unique(labels, return_counts=True)

  dominant = palette[np.argmax(counts)]
  return average,dominant

Using assumption that the first two layers in a CNN are for edge detection

In [14]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


model.features.denseblock1.denselayer1.conv1.register_forward_hook(get_activation('layer'))


In [ ]:
from torchvision.utils import save_image
import os
import pandas as pd
import torchray
from torchray.benchmark import plot_example


results = []
color=[]
device = get_device()
model.to(device).eval()
area = 0.1
start = True

for (x,y),z in tqdm(zip(dataloader,range(100))):  
  x = x.to(device)

  if y.item() == 2:
    # Extremal perturbation
    masks_1,_ = extremal_perturbation(
        model, x, y.item(),
        reward_func=contrastive_reward,
        perturbation="blur",
        debug=False,
        areas=[area],
        smooth=0.01,
        max_iter=800
    )
    ## Crop Important Area
    dat=torch.round(masks_1)[0].cpu().detach().numpy() # 'sharpen' mask
    true_points = np.argwhere(dat)
    if len(true_points) != 0:
      # take the smallest points and use them as the top left of your crop
      top_left = true_points.min(axis=0)
      # take the largest points and use them as the bottom right of your crop
      bottom_right = true_points.max(axis=0)

      ### Get Edge from Second Conv Layer ###
      output = model(x)
      edges = activation['layer'][0]
      ###
      
      x = x.squeeze()
      save_image(x, 'org.png')
      save_image(masks_1, 'mask.png')
      
      out = x[:,top_left[1]:bottom_right[1], 
                top_left[2]:bottom_right[2]]
      dim = [top_left,bottom_right]
      out =  torch.tensor(out, dtype=torch.float)
      save_image(out, 'img1.png')
      out = remove_background()
      # save_image(torch.tensor(out).permute(2,0,1).long(), 'no_bckgrd.png')
      plt.imshow(out)
      avg, dom = get_colors(out)
      out_tensor = torch.tensor(out, dtype = torch.float).permute(2,0,1)
      color.append(trans_color(out_tensor).cpu().detach().numpy())
      
      img_rep = torch.flatten(out_tensor, start_dim=1).cpu().detach().numpy()
      edge_rep = torch.flatten(edges, start_dim=0).cpu().detach().numpy()

      if start:
        src = out
        start = False
        results.append([*tuple(img_rep[0]),*tuple(img_rep[1]),*tuple(img_rep[2]), top_left[1],top_left[2],bottom_right[1],bottom_right[2], dom[0],dom[1],dom[2], 1, *tuple(edge_rep)])
      else:
        results.append([*tuple(img_rep[0]),*tuple(img_rep[1]),*tuple(img_rep[2]), top_left[1],top_left[2],bottom_right[1],bottom_right[2], dom[0],dom[1],dom[2], 1, *tuple(edge_rep)])
     

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


## Cluster Representations

Get Representations for Clustering

In [3]:
import numpy as np

#Sanity check
X = np.asarray(results)
print ("The shape of X is " + str(X.shape))

NameError: ignored

In [ ]:
# To perform PCA we must first change the mean to 0 and variance to 1 for X using StandardScalar
Clus_dataSet = StandardScaler().fit_transform(X) #(mean = 0 and variance = 1)

In [ ]:
from sklearn.decomposition import PCA
# Make an instance of the Model
variance = 2
pca = PCA(variance)

In [ ]:
#fit the data according to our PCA instance
pca.fit(Clus_dataSet)

In [ ]:
print("Number of components before PCA  = " + str(X.shape[1]))
print("Number of components after PCA 0.98 = " + str(pca.n_components_))

In [ ]:
#Transform our data according to our PCA instance
Clus_dataSet = pca.transform(Clus_dataSet)

In [ ]:
print("Dimension of our data after PCA  = " + str(Clus_dataSet.shape))

In [ ]:
#To visualise the data inversed from PCA
approximation = pca.inverse_transform(Clus_dataSet)
print("Dimension of our data after inverse transforming the PCA  = " + str(approximation.shape))

In [ ]:
from sklearn.metrics import silhouette_score

sil = []

ideal = None
max = float('-inf')
# x = Clus_dataSet

for k in range(5,10):
  kmeans = KMeans(init = "k-means++",n_clusters = k, n_init = 35, max_iter=1000).fit(X)
  labels = kmeans.labels_
  score = silhouette_score(X, labels, metric = 'euclidean')
  if(score > max):
    max = score
    ideal = kmeans

In [ ]:
k_means = ideal
print(k_means)

In [ ]:
k_means_labels = k_means.labels_ #List of labels of each dataset
print("The list of labels of the clusters are " + str(np.unique(k_means_labels)))

In [ ]:
k_means_cluster_centers = k_means.cluster_centers_ #numpy array of cluster centers
k_means_cluster_centers.shape #comes from 10 clusters and 420 features

In [ ]:
k_means.labels_

In [ ]:
#Find average "image" for each cluster
clusters = {}
for i,j in zip(k_means.labels_,color) :
  if i in clusters.keys():
    clusters.get(i).append(j)
    # clusters.update({i:clusters.get(i).append(j)})
  else:
    clusters.update({i:[j]})

## Use Chi^2 Similarity for Comparisons

In [ ]:
import cv2

def get_similar(arr):
  chosen = arr[0]
  x = {}
  for i in range(len(arr)):
    x.update({i: cv2.compareHist(np.array(chosen, dtype=np.float32), np.array(arr[i], dtype=np.float32), 2)})
  x = {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}
  return (x.keys())
    


In [ ]:
def choose_sim(arr,target,ratio):
  threshold = int(len(arr)*ratio)
  arr = arr[threshold:]
  result = []
  for i in range(len(target)):
    if(i in arr):
      result.append(target[i])
    
  return result

Find visualization by individual pixel comparisons

In [ ]:
# def get_individual_pixel_representation(arr):
#   li = []
#   for i in range(224):
#     li.append([])
#     for j in range(224):
#       dist = {0:0,1:0}
#       for img in arr:
#         pixel = img[0][i][j]
#         if pixel > 0:
#           pixel = 1
#         else:
#           pixel = 0
#         dist.update({pixel:dist.get(pixel)+1})
#       if(dist.get(0)>dist.get(1)):
#         li[i].append(0)
#       else:
#         li[i].append(1)
#   return li



Use "selective" averaging 

In [ ]:
# def selective_avg(arr,arr2):
#   for i in 


## Visualize Results

In [ ]:
import matplotlib.pyplot as plt
from scipy import ndimage
import cv2
import warnings

warnings.filterwarnings("ignore")

i = 0
texture = []
for key in clusters.keys():
  fig = plt.figure(figsize=(40,40))    
  ax = fig.add_subplot(8, 8, i + 1)
  arr = get_similar(clusters.get(key))
  arr = choose_sim(list(arr),clusters.get(key),0)
  arr = sum(arr)/len(arr)
  img = np.asarray(torch.tensor(arr).permute(1,2,0).cpu().squeeze().detach().numpy() ) ###HERE
  filter_blurred_f = ndimage.gaussian_filter(img, 1)
  alpha = 1
  sharpened = img + alpha * (img - filter_blurred_f)
  ax.imshow(sharpened, cmap = plt.cm.gray)
  # texture.append(sharpened)

  i += 1
plt.show()

# print("Texture Difference:",compare_texture(texture[0],texture[1]))


## Single Image High Resolution

In [ ]:
!git clone https://github.com/krasserm/super-resolution.git

In [ ]:
!cp -r super-resolution/model/ /content/
!cp super-resolution/utils.py /content/

In [ ]:
from model.srgan import generator
from utils import load_image, plot_sample
from model.common import resolve_single

model_hr = generator()
model_hr.load_weights('/content/drive/My Drive/Interpretability Experiments/srgan_weights/srgan/gan_generator.h5')

lr = load_image('img1.png')[:,:,:3]
sr = resolve_single(model_hr, lr)

plot_sample(lr, sr)
